**Essential imports:**

In [1]:
import random as r
import numpy as n
import math as m
from numpy.linalg import eig
import matplotlib.pyplot as p

**Calculation of $\pi$ using direct sampling:**

In [2]:
# Assume the radius = 1, and the center of the cirlce is the origin of the Cartesian coordinates.

n_trials = 40000
n_hits = 0

for i in range(n_trials):
    x, y = r.uniform(-1.0, 1.0), r.uniform(-1, 1)
    if x**2 + y**2 < 1: n_hits += 1 # r^2 = 1
        
print("Value of pi =", 4*n_hits/n_trials)

Value of pi = 3.1627


**Calculation of $\pi$ using direct sampling - multi-runs:**

In [3]:
def direct_pi(N):
    n_hits = 0
    for i in range(N):
        x, y = r.uniform(-1, 1), r.uniform(-1, 1)
        if x**2 + y**2 < 1: n_hits += 1
    return n_hits

n_runs = 1000
n_trials = 4000

for run in range(n_runs): obs = direct_pi(n_trials)/n_trials

**Calculation of $\pi$ using Markov chain sampling:**

In [ ]:
x, y = 1, 1 # Starting at the upper right corner of the square and assuming that the center of the cicle is the origin
delta = 0.1 # There are lower and upper limits to the extent of the throw.

# Does the convergence of the markov-chain Monte Carlo algorithm depend on the starting point and the value of delta?

n_trials = int(input("Number of trials: "))
n_hits = 0
for i in range(n_trials):
    del_x, del_y = r.uniform(-delta, delta), r.uniform(-delta, delta)
    if abs(x + del_x) < 1 and abs(y + del_y) < 1: # That is, if the pebble is thrown inside the square... Note that the trial is counted anyway.
        x, y = x + del_x, y + del_y
    if x**2+y**2 < 1: n_hits += 1 # That is, if the pebble is inside the circle... Rejections inside the circle are still counted as hits.
print("Value of pi:", 4*n_hits/n_trials)

**Calculation of $\pi$ using Markov chain sampling - multi-runs:**

In [ ]:
def markov_pi(N, delta):
    x, y = 1, 1
    n_hits = 0
    for i in range(N):
        del_x, del_y = r.uniform(-delta, delta), r.uniform(-delta, delta)
        if abs(x + del_x) < 1 and abs(y + del_y) < 1:
            x, y = x + del_x, y + del_y
        if x**2 + y**2 < 1.0: n_hits += 1
    return n_hits

n_runs = int(input("Num of runs = "))
n_trials = int(input("Num of trials = "))
delta = float(input("Value of delta = "))
for run in range(n_runs): print(4*markov_pi(n_trials, delta)/n_trials)

**Pebble game, discrete Markov-chain sampling and the use of the neighbor table:**

In [ ]:
neighbor = [[1, 3, 0, 0], # The current site is just the raw number.
            [2, 4, 0, 1],
            [2, 5, 1, 2],
            [4, 6, 3, 0],
            [5, 7, 3, 1],
            [5, 8, 4, 2],
            [7, 6, 6, 3],
            [8, 7, 6, 4],
            [8, 8, 7, 5]]

# Numbering scheme of the pepple game box:

#   6   7   8
#   3   4   5
#   0   1   2

t_max = int(input("Number of desired moves = "))
site = int(input("Current site = "))
t = 0 # Actually, this is discretized time!
while t < t_max:
    t = t + 1
    site = neighbor[site][r.randint(0, 3)] # The next site is chosen randomly. randint(a, b) generates a random integer bet. a & b inclusive.
    print(site)

**Pebble game with inhomogenous transition probability:**

In [ ]:
hist = [0, 0, 0, 0, 0, 0, 0, 0, 0]

neighbor =  [[1, 3, 0, 0],
             [2, 4, 0, 1],
             [2, 5, 1, 2],
             [4, 6, 3, 0],
             [5, 7, 3, 1],
             [5, 8, 4, 2],
             [7, 6, 6, 3],
             [8, 7, 6, 4],
             [8, 8, 7, 5]]

weight = [3.0, 0.5, 1.0, 0.5, 1.0, 0.5, 2.0, 0.5, 1.0] # Statistical weights of configs, not probs.
pos = 8
n_iterations = int(input("Number of iterations: "))

for i in range(n_iterations):
    new_pos = neighbor[pos][r.randint(0, 3)]
    if r.random() < weight[new_pos]/weight[pos]: pos = new_pos # MOST IMPORTANT LINE IN THE CODE!
    hist[pos] += 1

norm = sum(weight)
for k in range(9): print('Site:\t', k, '\tWeight:\t', weight[k], '\tHistogram:\t', norm*hist[k]/n_iterations)

**Performing data bunching on a Markov-chain generated dataset (experimental code):**

In [ ]:
def markov_pi(N, d):
    x, y = 1, 1
    data = []
    for i in range(N):
        dx, dy = r.uniform(-d, d), r.uniform(-d, d)
        if abs(x + dx) < 1 and abs(y + dy) < 1: x, y = x + dx, y + dy
        if x**2+y**2 < 1: data.append(4)
        else: data.append(0)
    return data

n_trials = 1000
delta = 0.1
data = markov_pi(n_trials, delta)

sum1 = 0
sum2 = 0
N = int(len(data)/2)
y = []
for i in range(N):
    # print("data[2i]\t", str('% 0.4f' % data[2*i]).replace("'", ""), "\tdata[2i+1]\t", str('% 0.4f' % data[2*i+1]).replace("'", ""))
    sum1 += data[2*i] + data[2*i+1]
    sum2 += data[2*i]**2 + data[2*i+1]**2
    y.append((data[2*i] + data[2*i+1])/2)

error = m.sqrt(sum2/(2*N) - (sum1/(2*N))**2)/m.sqrt(2*N)
print("Sum/2N:", "% 0.6f" % (sum1/(2*N)), "\tError:", "% 0.6f" % error)
# print("\n"+str(["% 0.4f" % i for i in y]).replace("'", ""))

**Generating the transfer matrix from the neighbor table, and then generating the probability vectors from the transfer matrix:**

In [ ]:
neighbor =  [[1, 3, 0, 0],
             [2, 4, 0, 1],
             [2, 5, 1, 2],
             [4, 6, 3, 0],
             [5, 7, 3, 1],
             [5, 8, 4, 2],
             [7, 6, 6, 3],
             [8, 7, 6, 4],
             [8, 8, 7, 5]]

transfer = n.zeros((9, 9))

# Filling the transfer matrix:
for i in range(9):
    for j in range(4): transfer[neighbor[i][j], i] += 0.25

n.set_printoptions(formatter={'float': '{:0.2f}'.format})

print("Transfer matrix:\n")
for row in range(transfer.shape[0]): print(transfer[row, :])

print("\nProbabiliy vectors:\n")

n.set_printoptions(formatter={'float': '{:0.5f}'.format})

# Constructing the probability vectors:
prob_vec = n.zeros(9)
prob_vec[8] = 1.0

for t in range(100):
    print(t, "\t", prob_vec, "\n")
    prob_vec = n.matmul(transfer, prob_vec)

**Finding eigenvalues and eigenvectors of the transfer matrix**

In [ ]:
from numpy.linalg import eig

eigenvalues, eigenvectors = eig(transfer)
n.set_printoptions(formatter={'float': '{: 0.4f}'.format})
for i in range(9): print("% 0.2f" % (eigenvalues[i]), eigenvectors[i])

**Escaping reducibility in Markov-chain Monte Carlo**

In [ ]:
transfer = n.zeros((18, 18))

for i in range(9):
    for j in range(4): 
        transfer[neighbor[i][j], i] += 0.25     # Red pebble game
        transfer[neighbor[i][j]+9, i+9] += 0.25 # Blue pebble game   

# Small transition epsilon between red 2 and blue 6
epsilon = 0.04

transfer[6+9, 2] = epsilon
transfer[2, 6+9] = epsilon

# This prob. has to be taken from the not-moving prob. so that prob. sum = 1.
transfer[2, 2] -= epsilon    
transfer[6+9, 6+9] -= epsilon

eigenvalues, eigenvectors = eig(transfer)

n.set_printoptions(formatter={'complexfloat': '{: .4f}'.format})
for i in eigenvalues: print('{: .4f}'.format(i))

**Escaping periodicity (i.e., recurrence) in Markov-chain Monte Carlo**

In [ ]:
eps = 0.01
transfer = n.array([[eps,   1-eps],
                    [1-eps, eps  ]])
eigenvalues, eigenvectors = eig(transfer)
for i in eigenvalues: print('{: .4f}'.format(i))

**The following code is wrong because it is sampling two random numbers:**

In [ ]:
N = 20
position = 0
for t in range(10):
    if r.uniform(0, 1) < 0.5: position = (position + 1) % N
    elif r.uniform(0, 1) > 0.5: position = (position - 1) % N
    print(position)

**In direct sampling, the standard deviation decreases as the number of trials is increased:**

In [ ]:
def direct_pi(N):
    n_hits = 0
    for i in range(N):
        x, y = r.uniform(-1, 1), r.uniform(-1, 1)
        if x**2+y**2 < 1.0:
            n_hits += 1
    return n_hits

n_runs = 500
n_trials_list = []
sigmasq_list = []

for i in range(4, 13):
    n_trials = 2**i
    sigmasq = 0.0
    for run in range(n_runs):
        pi_est = 4*direct_pi(n_trials)/n_trials
        sigmasq += (pi_est - m.pi)**2
    sigmasq_list.append(m.sqrt(sigmasq/(n_runs)))
    n_trials_list.append(n_trials)

p.plot(n_trials_list, sigmasq_list, 'o')
p.xscale('log')
p.yscale('log')
p.xlabel('Number of trials')
p.ylabel('Root mean square deviation')
p.title('Direct sampling of $\pi$: root mean square deviation vs. number of trials')
p.savefig('direct_sampling_rms_deviation.png')
p.show()

**Choosing $\delta$ that gives an acceptance rate approx. equal to 0.5:**

In [ ]:
x, y = 1.0, 1.0
delta = [0.062, 0.125, 0.25, 0.5, 1.0, 2.0, 4.0]

n_trials = 2**12
n_hits = 0
accept_rate = 0
for d in delta:
    for i in range(n_trials):
        del_x, del_y = r.uniform(-d, d), r.uniform(-d, d)
        if abs(x + del_x) < 1.0 and abs(y + del_y) < 1.0:
            x, y = x + del_x, y + del_y
            accept_rate += 1
        if x**2+y**2 < 1.0: n_hits += 1
    print("delta\t", d,"\tAcceptance rate:\t", n_hits/n_trials)

In [ ]:
x, y = 1.0, 1.0
delta = [0.062, 0.125, 0.25, 0.5, 1.0, 2.0, 4.0]

n_trials = 2**12
n_hits = 0

print("delta\t|\t Acceptance rate")
for d in delta:
    accept_rate = 0
    for i in range(n_trials):
        del_x, del_y = r.uniform(-d, d), r.uniform(-d, d)
        if abs(x + del_x) < 1.0 and abs(y + del_y) < 1.0:
            x, y = x + del_x, y + del_y
            accept_rate += 1
    print(d, "\t|\t", accept_rate/n_trials)

**Defining `obs` to generate a 1-D Markov-chain generated dataset that is suitable for data bunching:**

In [ ]:
n_trials = 400000
n_hits = 0
obs_sum = 0
obs2_sum = 0

for i in range(n_trials):
    x, y = r.uniform(-1, 1), r.uniform(-1, 1)
    obs = 0
    if x**2+y**2 < 1:
        n_hits += 1
        obs = 4 # "obs" is a Bernoulli variable that takes either 0 or 4.
    obs_sum += obs
    obs2_sum += obs**2

obs_av = obs_sum/n_trials
obs2_av = obs2_sum/n_trials
var = obs2_av - obs_av**2
sdev = m.sqrt(var)

print('Obs av:\t\t{0:.4f}\nObs^2 av:\t{1:.4f}\nVariance:\t{2:.4f}\nStandard dev:\t{3:2.4f}'.format(obs_av, obs2_av, var, sdev))

**Data bunching for a Markov chain:**

In [ ]:
# Generating 1-D dataset for a 2-D problem: 

def markov_pi(N, d):
    x, y = 1, 1
    data = []
    for i in range(N):
        dx, dy = r.uniform(-d, d), r.uniform(-d, d)
        if abs(x + dx) < 1 and abs(y + dy) < 1: x, y = x + dx, y + dy
        if x**2+y**2 < 1: data.append(4)
        else: data.append(0)
    return data

k = 20
n_trials = 2**k
delta = 0.1
data = markov_pi(n_trials, delta)
errors  = []
bunches = []

for i in range(k):
    new_data = []
    mean = 0
    mean_sq = 0
    N = len(data)
    while data != []:
        x = data.pop()
        y = data.pop()
        mean += x + y
        mean_sq += x**2 + y**2
        new_data.append((x + y)/2)
    errors.append(m.sqrt(mean_sq/N - (mean/N)**2)/m.sqrt(N))
    bunches.append(i)
    data = new_data[:]
    print(abs(m.pi - mean/N))

p.plot(bunches, errors, '--s')
p.xlabel('Iteration')
p.ylabel('Apparent error')
p.title('Bunching: naive error vs. iteration number')
p.grid("on")
p.show()

**Bayesian Monte Carlo:**

In [ ]:
n_hits = 0

for k in range(10):
    while n_hits != 3156: 
        pi_test = r.uniform(0, 4)
        n_hits = 0
        for i in range(4000):
            if r.uniform(0, 1) < pi_test/4: n_hits += 1
    print(pi_test)